In [4]:
import pandas as pd
import re
import numpy as np

In [5]:
gaon_df = pd.read_csv('../data/gaon_final_data.csv', index_col=0)


In [6]:
melon_df = pd.read_csv('../data/melon_final_data.csv', index_col=0)

### 1.데이터 특수문자 공백제거

In [7]:
# 괄호 따로 저장해서 컬럼으로 추가
### 가수 괄호는 따로 컬럼에 저장 *혹시 데이터 일치여부에 사용할 수 있으니

def Save_Bracket(df, column):
    result = []
    new_column = []
    for i in range(len(df)):
        data = df.iloc[i][f'{column}']

        # 소문자로 바꾸기
        data = data.lower()

        # 괄호 안 내용 저장
        if data.find('(') != -1:
            p = re.compile('\(([^)]+)')
            data_bracket = p.findall(data)[0]
            # 괄호 삭제
            data = re.sub(' \([^)]*\)','', data)

            # 리스트에 데이터 추가
            result.append(data)
            new_column.append(data_bracket)
            
        else:
            result.append(data)
            new_column.append(np.NaN)

    df[f'{column}'] = result
    df[f'{column} bracket'] = new_column

    return df

# 멜론 데이터 전처리
def Make_Clean_Name(df, column):
    result = []
    for i in range(len(df)):
        data = df.iloc[i][f'{column}']

        # 특수문자 제거 (콤마는 빼고 제거)
        data = re.sub('[-=+#/\?:^.@★*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', " ",data)

        # 소문자로 바꾸기
        data = data.lower()
        
        if column == 'singer':
            # 가수이름은 모든 공백 제거
            data = data.replace(' ','')
        else:  
            # 앞,뒤 공백 제거
            data = data.strip()

        result.append(data)

    df[f'{column}'] = result

    return df


In [8]:
# gaon_df 전처리
gaon_df = Save_Bracket(gaon_df, 'music')
gaon_df = Save_Bracket(gaon_df, 'singer')
gaon_df = Make_Clean_Name(gaon_df, 'music')
gaon_df = Make_Clean_Name(gaon_df, 'singer')



In [9]:
# melon_df 전처리
melon_df = Make_Clean_Name(melon_df, 'music')
melon_df = Make_Clean_Name(melon_df, 'singer')

### 2.차트에 든 횟수 DV(종속변수) 컬럼 만들기

In [10]:
gaon_df['singer-music'] = gaon_df['singer'] + '-' + gaon_df['music']
DV_df = pd.DataFrame(gaon_df['singer-music'].value_counts())
DV_df.reset_index(drop=False, inplace=True)
DV_df.rename(columns={'index':'music', 'music':'rank_count'}, inplace=True)

In [11]:
DV_df

,music,singer-music
0,방탄소년단-봄날,324
1,"폴킴-모든 날, 모든 순간",268
2,"akmu-어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",171
3,아이유-밤편지,169
4,임영웅-이제 나만 믿어요,161
...,...,...
7369,"백지영,나원주-니가 내리는 날",1
7370,에이핑크-동화 같은 사랑,1
7371,에이핑크-천사가 아냐,1
7372,싸이-rock will never die,1


### 3.멜론에 가온 start_date 붙이기

In [12]:
def Add_startdate(df1, df2):
    date_list = []
    for i in range(len(df1)):

        music = df1['music'].iloc[i]
        singer = df1['singer'].iloc[i]

        try:
            start_date = df2['start_date'].loc[(df2['music']==music) & ((df2['singer']==singer)|(df2['singer bracket']==singer))].iloc[0]
            date_list.append(start_date)
        except:
            date_list.append(np.NaN)

    df1['start_date'] = date_list

    return df1

In [13]:
melon_df = Add_startdate(melon_df, gaon_df)

In [14]:
melon_df['start_date'].isnull().sum()

425

#### start,release로 7일이내 순위진입 T,F 뽑아서 컬럼

In [15]:
# 날짜 datetime 형식으로 변환환
melon_df['start_date'] = pd.to_datetime(melon_df['start_date'])
melon_df['release_date'] = pd.to_datetime(melon_df['release_date'])

# 7일 계산해서 calculate컬럼으로 추가
melon_df['calculate'] = melon_df['release_date'] - melon_df['start_date']
# 7일이내면 값 1, 아니면 0
melon_df['within 7days'] =  abs(melon_df['calculate']) < pd.Timedelta(days=7)
melon_df['within 7days'].replace({True:1, False:0}, inplace=True)

# 쓸모없는 컬럼 삭제
melon_df.drop(columns=['start_date', 'calculate'], inplace=True)

### 4.melon에 카테고리에서 국내드라마로 OST 컬럼 이진변수 만들기

In [16]:
# ost컬럼 추가 + 카테고리에서 발라드,국내드라마에서 국내드라마는 삭제
def Make_OST_column(df):
    ost_list = []
    category_list = []

    for i in range(len(df)):
        category = df['category'].iloc[i]
        try:
            a, b = category.split(', ')
            if b == '국내드라마':
                ost_list.append(1)
                category_list.append(a)
            else:
                ost_list.append(0)
                category_list.append(category)
        except:
            ost_list.append(0)
            category_list.append(category)

    df['ost'] = ost_list
    df['category'] = category_list

    return df

In [17]:
melon_df = Make_OST_column(melon_df)

In [18]:
melon_df['ost'].value_counts()

0    4447
1     459
Name: ost, dtype: int64

### 5.가수 이름 word2vec 


In [19]:
from gensim.models.word2vec import Word2Vec

In [20]:
m_singer_list = melon_df['singer'].tolist()

In [21]:
#모델 불러오기
#리스트인 melon_singer를 word2vec , mincount 1로 1개이상 값은 다 

model = Word2Vec(sentences=[m_singer_list], min_count=1)

singer_w2_dict = {}
for singer in m_singer_list:
    singer_w2_dict[singer] = model.wv[singer]


### 6.melon 카테고리에서 자른것들 원핫인코딩

In [22]:
# # 카테고리 세분화 - 2개까지만
# def Divide_category(df):
#     cate_1 = []
#     cate_2 = []


#     for i in range(len(df)):
#         category = df['category'].iloc[i]
#         category_list = category.split(',')
#         if len(category_list) == 1:
#             cate_1.append(category_list[0])
#             cate_2.append(np.NaN)
#         elif len(category_list) >= 2:
#             cate_1.append(category_list[0])
#             cate_2.append(category_list[1])
        
#     df['category_1'] = cate_1
#     df['category_2'] = cate_2

#     return df

# 카테고리 맨 앞꺼 빼고 삭제하는 함수
def delete_category(df):
    cate = []

    for i in range(len(df)):
        category = df['category'].iloc[i]
        category_list = category.split(',')
        cate.append(category_list[0])

    df['category'] = cate

    return df

#### 6-2 카테고리 원핫인코딩

In [23]:
melon_df = delete_category(melon_df)

In [24]:
melon_df[['category']]

,category
0,댄스
1,발라드
2,발라드
3,발라드
4,포크/블루스
...,...
4901,댄스
4902,발라드
4903,발라드
4904,댄스


In [25]:
from sklearn.preprocessing import OneHotEncoder

In [26]:
ohe = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
train_cat = ohe.fit_transform(melon_df[['category']])
train_cat

c:\Users\bin\anaconda3\envs\shinhanAI\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
train_cat[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0.])